In [ ]:
from torch.utils.data import DataLoader
from torchgeo.datasets import VHR10
 
dataset = VHR10(root="...", download=True, checksum=True)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4)
 
for batch in dataloader:
    image = batch["image"]
    label = batch["label"]
 
    # train a model, or make predictions using a pre-trained model

In [4]:
import os

# 定义文件夹路径
folder_path = "/root/workspace/ctf53sc7v38s73e0mksg/maps/new_qd_years"

# 定义原始字符串和目标字符串
original_suffix = "@116.35551452636700@40.09815882135800@116.44632339477501@40.15118932709900@.tif"
new_suffix = "@116.35551452636700@40.15118932709900@116.44632339477501@40.09815882135800@.tif"

# 遍历文件夹中的所有文件
for filename in os.listdir(folder_path):
    # 检查文件名是否以原始字符串结尾
    # if filename.endswith(original_suffix):
        # 构造新的文件名
    new_filename = filename.replace(original_suffix, new_suffix)
        
    # 构造完整的文件路径
    old_file_path = os.path.join(folder_path, filename)
    new_file_path = os.path.join(folder_path, new_filename)
    
    # 重命名文件
    os.rename(old_file_path, new_file_path)
    print(f"文件 {filename} 已重命名为 {new_filename}")


文件 @20120905@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif 已重命名为 @20120905@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif
文件 @20131005@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif 已重命名为 @20131005@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif
文件 @20170319@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif 已重命名为 @20170319@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif
文件 @20171027@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif 已重命名为 @20171027@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif
文件 @20191107@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif 已重命名为 @20191107@120.40747060326400@36.60902770136900@120.46053192297900@36.58563292439400@.tif
文件 @20211104@120.40747060326400@36.60902770136900@120.46053192297

In [ ]:
import torch

# assume 'x' is your input tensor with shape (32, 1, 2560)
x = torch.randn(32, 1, 2560)

# perform PCA on the input tensor
components, explained_variance = torch.pca_lowrank(x, q=1280)

# reconstruct the reduced tensor
x_reduced = torch.matmul(x, components)

print(x_reduced.shape)  # should be (32, 1, 1280)

In [ ]:
import torch

# assume 'x' is your input tensor with shape (32, 1, 2560)
x = torch.randn(32, 1, 2560)

# perform SVD (PCA) on the input tensor
u, s, v = torch.svd(x)

# select the top 1280 singular values and corresponding singular vectors
u_reduced = u[:, :, :1280]
s_reduced = s[:, :1280]
v_reduced = v[:, :1280, :]

# reconstruct the reduced tensor
x_reduced = torch.matmul(u_reduced, torch.diag_embed(s_reduced))
print(x_reduced.shape)  # should be (32, 1, 1280)
x_reduced = torch.matmul(x_reduced, v_reduced)

print(x_reduced.shape)  # should be (32, 1, 1280)

In [ ]:
import torch

def pca_reduction(data, n_components):
    # 标准化数据
    data_mean = torch.mean(data, dim=0, keepdim=True)
    data_std = torch.std(data, dim=0, keepdim=True)
    data_normalized = (data - data_mean) / data_std

    # 计算协方差矩阵
    cov_matrix = torch.cov(data_normalized.view(-1, data.size(-1)).T)

    # 计算特征值和特征向量
    eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix)

    # 选择前 n_components 个特征向量
    top_eigenvectors = eigenvectors[:, -n_components:]

    # 投影数据
    reduced_data = torch.matmul(data_normalized.view(-1, data.size(-1)), top_eigenvectors)

    # 调整形状为 (32, 1, 100)
    reduced_data = reduced_data.view(data.size(0), data.size(1), n_components)

    return reduced_data

# 假设你有一个形状为 (32, 1, 240) 的张量
input_tensor = torch.randn(32, 1, 240)

# 进行 PCA 降维到 100 维
output_tensor = pca_reduction(input_tensor, n_components=100)

print(output_tensor.shape)  # 输出应为 torch.Size([32, 1, 100])

In [1]:
command = 'train_dc.py -bb ResNet50 -agg MixVPR --device cuda --num_workers 16 --batch_size 64 --dataset_name ct01 --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-100-700-seperate-height --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-cities-test -ltc 3 4 -ltf 1 -ipe 2000 --N 2 --fft --fft_log_base 1.5'

parts = command.split()
result = ', '.join([f'"{part}"' for part in parts])
print(result)

"train_dc.py", "-bb", "ResNet50", "-agg", "MixVPR", "--device", "cuda", "--num_workers", "16", "--batch_size", "64", "--dataset_name", "ct01", "--train_set_path", "/root/workspace/ctf53sc7v38s73e0mksg/maps/HC-100-700-seperate-height", "--test_set_path", "/root/workspace/ctf53sc7v38s73e0mksg/maps/HC-cities-test", "-ltc", "3", "4", "-ltf", "1", "-ipe", "2000", "--N", "2", "--fft", "--fft_log_base", "1.5"


In [3]:
command = 'train_dc.py -bb ResNet50 -agg MixVPR --device cuda --num_workers 16 --batch_size 64 --dataset_name ct01 --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-100-700-seperate-height --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-cities-test -ltc 3 4 -ltf 1 -ipe 2000 --N 2 --fft --fft_log_base 1.5'
command = 'he_train.py --M 300 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 336 448 --test_resize 336 --backbone radio_v2.5-h --train_blocks_num 2 --return_token --threshold 300 --num_workers 16 --device cuda --exp_name ct01-5years --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/ct01_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/ct01-test-125 --dataset_name ct01-5years'

command = "--threshold 30 --device cuda --exp_name ct01_csv --test_set_list ct01 --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-cities-test --test_resize 336"

command = '-bb ResNet50 -agg MixVPR --device cuda --num_workers 16 --batch_size 64 --dataset_name 2022 2013 --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-100-700-seperate-height --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/qd -ltc 3 4 -ltf 1 -ipe 2000 --N 2'

command = 'train_dc.py -bb ResNet50 -agg MixVPR --device cuda --num_workers 16 --batch_size 64 --dataset_name 2022 qd_train --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/HC-100-700-seperate-height --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/qd -ltc 3 4 -ltf 1 -ipe 2000 --N 2'

command = '--test_resize 320 320 --device cuda --exp_name ct01-6years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/ct01_125_years/202306_125 --dataset_name ct01-6years -csv /root/workspace/ctf53sc7v38s73e0mksg/code/Height-Classify/logs/hc-ResNet50-MixVPR/2025-01-15_14-03-32/results.csv --test_dataset_name HC-ct01'

command = '--M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 336 448 --test_resize 336 --backbone radio_v2.5-h --train_blocks_num 2 --return_token --threshold 100 --num_workers 16 --device cuda --exp_name qd-6years-trace1 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years/2022_125 --dataset_name qd-6years --resume_model /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/logs/qd-6years-train-M100/2025-01-15_22-40-46/best_model.pth -csv /root/workspace/ctf53sc7v38s73e0mksg/code/Height-Classify/logs/hc-ResNet50-MixVPR/2025-01-17_11-16-20/results.csv --test_dataset_name HC-qd1'

command = 'he_train.py --M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 336 448 --test_resize 336 --backbone radio_v2.5-h --train_blocks_num 2 --aggregator MixVPR --threshold 200 --num_workers 16 --device cuda --exp_name qd-8years-train-M100 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/tmp_img/qd_test_cut --dataset_name qd-8years --classifier_type QAMC --test_dataset_name trace1_ct'

command = 'he_train.py --M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 336 448 --test_resize 336 --backbone EfficientNet_B5 --aggregator MixVPR --threshold 100 --num_workers 16 --device cuda --exp_name qd-8years-train-M100 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/tmp_img/qd_test_cut --dataset_name qd-8years --classifier_type QAMC --test_dataset_name trace1_ct'

command = 'save_descriptor.py --M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 336 448 --test_resize 336 --backbone EfficientNet_B5 --aggregator MixVPR --threshold 100 --num_workers 16 --device cuda --exp_name qd-8years-train-M100 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/tmp_img/qd_test_cut --dataset_name qd-8years --classifier_type QAMC --test_dataset_name trace1_ct'

command = "python he_output_cut.py --test_resize 224 224 --device cuda --exp_name ct01-6years --dataset_name ct01-6years --num_workers 8 -csv /root/workspace/ctf53sc7v38s73e0mksg/code/Height-Classify/logs/hc-ResNet50-MixVPR/2025-01-15_14-03-32/results.csv --test_dataset_name HC-ct01 --model AnyLoc"

command = 'he_train.py --M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 224 224 --test_resize 224 224 --backbone dinov2_vitb14 --aggregator CricaVPR --threshold 100 --num_workers 16 --device cuda --exp_name Baseline-qd-8years-train-M100 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/tmp_img/qd_test_cut --dataset_name qd-8years --classifier_type QAMC --test_dataset_name trace_ct'

command = "--M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 224 224 --test_resize 224 224 --backbone dinov2_vitb14 -ds finetune --aggregator salad --threshold 100 --num_workers 16 --device cuda --exp_name Baseline-qd-8years-train-M100 --train_set_path /root/workspace/ctvsuas7v38s73eo9qlg/maps/qd_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/tmp_img/qd_test_cut --dataset_name qd-8years --classifier_type QAMC --test_dataset_name trace_ct"

command = '--M 100 --N 2 --min_images_per_class 20 --iterations_per_epoch 2000 --batch_size 64 --train_resize 320 320 --test_resize 320 320 --backbone ResNet50 --aggregator MixVPR --threshold 100 --num_workers 16 --device cuda --exp_name ct01-6years-test --train_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/ct01_125_years --test_set_path /root/workspace/ctf53sc7v38s73e0mksg/maps/ct01-test-125_new --dataset_name ct01-6years-MixVPR --classifier_type QAMC --test_dataset_name ct01-ct -csv /root/workspace/ctf53sc7v38s73e0mksg/code/Height-Classify/logs/hc-ResNet50-MixVPR/2025-01-15_14-03-32/results.csv --resume_model /root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/logs/Baseline-ct01-6years-train-M100/2025-03-26_21-00-58/best_model.pth --model_classifier_layer'

parts = command.split()
formatted_parts = []
for part in parts:
    if part.startswith("--") or part.startswith("-"):
        formatted_parts.append("\n" + f'"{part}"')
    else:
        formatted_parts.append(f'"{part}"')

result = ", ".join(formatted_parts)
print(result)


"--M", "100", 
"--N", "2", 
"--min_images_per_class", "20", 
"--iterations_per_epoch", "2000", 
"--batch_size", "64", 
"--train_resize", "320", "320", 
"--test_resize", "320", "320", 
"--backbone", "ResNet50", 
"--aggregator", "MixVPR", 
"--threshold", "100", 
"--num_workers", "16", 
"--device", "cuda", 
"--exp_name", "ct01-6years-test", 
"--train_set_path", "/root/workspace/ctf53sc7v38s73e0mksg/maps/ct01_125_years", 
"--test_set_path", "/root/workspace/ctf53sc7v38s73e0mksg/maps/ct01-test-125_new", 
"--dataset_name", "ct01-6years-MixVPR", 
"--classifier_type", "QAMC", 
"--test_dataset_name", "ct01-ct", 
"-csv", "/root/workspace/ctf53sc7v38s73e0mksg/code/Height-Classify/logs/hc-ResNet50-MixVPR/2025-01-15_14-03-32/results.csv", 
"--resume_model", "/root/workspace/ctf53sc7v38s73e0mksg/code/UAV-Divide-Classify/logs/Baseline-ct01-6years-train-M100/2025-03-26_21-00-58/best_model.pth", 
"--model_classifier_layer"
